In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from category_encoders import TargetEncoder
import matplotlib.pyplot as plt


Load & check data

In [3]:
train = pd.read_csv("./data/train.csv")

train.dtypes

id                        int64
annual_income           float64
debt_to_income_ratio    float64
credit_score              int64
loan_amount             float64
interest_rate           float64
gender                   object
marital_status           object
education_level          object
employment_status        object
loan_purpose             object
grade_subgrade           object
loan_paid_back          float64
dtype: object

Encoding categorical data

In [24]:
def categorical_data_values(df):
    for i in df.columns:
        if df[i].dtype == 'object':
            print(f"column: {i}: values: {df[i].unique()}")
        else:
            pass

In [25]:
categorical_data_values(train)

column: gender: values: ['Female' 'Male' 'Other']
column: marital_status: values: ['Single' 'Married' 'Divorced' 'Widowed']
column: education_level: values: ['High School' "Master's" "Bachelor's" 'PhD' 'Other']
column: employment_status: values: ['Self-employed' 'Employed' 'Unemployed' 'Retired' 'Student']
column: loan_purpose: values: ['Other' 'Debt consolidation' 'Home' 'Education' 'Vacation' 'Car'
 'Medical' 'Business']
column: grade_subgrade: values: ['C3' 'D3' 'C5' 'F1' 'D1' 'D5' 'C2' 'C1' 'F5' 'D4' 'C4' 'D2' 'E5' 'B1'
 'B2' 'F4' 'A4' 'E1' 'F2' 'B4' 'E4' 'B3' 'E3' 'B5' 'E2' 'F3' 'A5' 'A3'
 'A1' 'A2']


In [20]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
id,593994.0,296996.500000,171471.442236,0.000,148498.250,296996.500,445494.750,593993.000
annual_income,593994.0,48212.202976,26711.942078,6002.430,27934.400,46557.680,60981.320,393381.740
debt_to_income_ratio,593994.0,0.120696,0.068573,0.011,0.072,0.096,0.156,0.627
credit_score,593994.0,680.916009,55.424956,395.000,646.000,682.000,719.000,849.000
loan_amount,593994.0,15020.297629,6926.530568,500.090,10279.620,15000.220,18858.580,48959.950
interest_rate,593994.0,12.356345,2.008959,3.200,10.990,12.370,13.680,20.990
loan_paid_back,593994.0,0.798820,0.400883,0.000,1.000,1.000,1.000,1.000


In [ ]:
def plot_numeric_columns(df,columns_to_exclude):
    numeric_dfs = df.select_dtypes(include="number")
    numeric_dfs = numeric_dfs.drop(columns=[c for c in columns_to_exclude if c in numeric_dfs.columns])
    numeric_dfs.hist(figsize=(10, 6))
    plt.tight_layout()
    plt.show()

In [31]:
plot_numeric_columns(train,columns_to_exclude=['id','loan_paid_back'])

KeyError: "['id', 'loan_paid_back'] not found in axis"

In [ ]:
X_train = train.drop(columns=['loan_paid_back'])
y_train = train['loan_paid_back'] 

In [ ]:
# Define encoder
te = TargetEncoder(
    cols=['education_level', 'employment_status', 'loan_purpose', 'grade_subgrade'],
    smoothing=10
)

# Prepare out-of-fold encoded training data
kf = KFold(n_splits=5, shuffle=True, random_state=42)
X_train_enc = X_train.copy()

for train_idx, val_idx in kf.split(X_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr = y_train.iloc[train_idx]
    
    te.fit(X_tr, y_tr)
    X_val_enc = te.transform(X_val)
    
    X_train_enc.loc[X_val.index, te.cols] = X_val_enc[te.cols]

# Fit final encoder on full training data
te.fit(X_train, y_train)

# Transform test data (assuming you have X_test already)
X_test_enc = te.transform(X_test)

NameError: name 'X_test' is not defined